<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Neural Networks

## *Data Science Unit 4 Sprint 2 Assignment 1*

## Define the Following:
You can add image, diagrams, whatever you need to ensure that you understand the concepts below.

### Input Layer:

* aka x = dependent variable
* Each input is a feature or category from the dataframe
* The more inputs we have the bigger the input layer (more nodes)



### Hidden Layer:

* A layer within the NN that allows for various models to be held. We don't directly interact with the hidden layer.
* The more complicated a Hidden Layer, the more time and resource intensive the NN will be. 
* Hidden Layers can be thought of as a series of models we pass the data through for further processing.

### Output Layer:

* The final layer of the Neural Network, where we receive our predictions
* Illustrated as a scientific number currently
* At this step the NN is feeding the finalized predictions from the models in the wires/hidden layers
    Then the NN finds the best of the best and releases those, feeding them back to the input layer for comparison and update.

### Neuron:

* Also known as a node, basically an encapsulated feature, with a minimum threshold for firing a signal to the next node. If the signal is weaker, less precedence. If the threshold is met and exceeded, that node will be considered a more superior node. 

### Weight:

* Based on the accuracy of the current model on the node/category. If more accurate, or it reaches the threshold for firing a signal, the node's weight is likely to be higher, whereas a node that never sends a signal would be considered negative (or, in simple terms, something for the NN to ignore.*

### Activation Function:

* The function used to calculate the point of activation for the node to fire to the next layer of nodes. 

### Node Map:

* Showcases the layout of the neural network. Different node maps are called different things, depending on presence of inputs, outputs, hidden layers. 

### Perceptron:

* The most basic NN model. Requires two inputs, has one output, no showcased hidden layers. Weights are the wires connecting the perceptron model, bias is assumed to be accounted for in most node maps. 

## Inputs -> Outputs

### Explain the flow of information through a neural network from inputs to outputs. Be sure to include: inputs, weights, bias, and activation functions. How does it all flow from beginning to end?

Each node is a category in a datafram. That node is then passed through the activation function. If the prediction and subsequent comparison of the target within the node is accurate, the node gets a higher weight. This is done using the MAE/MSE model in our example. Whichever metric you use, the better the prediction is to the target, the more likely that node is going to fire to the next layer of modes. After applying the weight, we then apply the bias (the average mean of the category encapsulated as a node in this case), and move on to the next layer. In the Perceptron model, the inputs move through the activation function/weight/bias, and then are output as the amount the prediction is closest to the actual target. (scientific notation). We then pass that information back to the input mode in order to improve the weights. 

## Write your own perceptron code that can correctly classify (99.0% accuracy) a NAND gate. 

| x1 | x2 | y |
|----|----|---|
| 0  | 0  | 1 |
| 1  | 0  | 1 |
| 0  | 1  | 1 |
| 1  | 1  | 0 |

In [64]:
import pandas as pd
import numpy as np
np.random.seed(143)
data= { 'x1': [0,1,0,1],
         'x2': [0,0,1,1],
         'y':  [1,1,1,0]
       }

inputs = [[0,1,0,1], [0,0,1,1]]
correct_output = [1,1,1,0]

df = pd.DataFrame.from_dict(data).astype('int')
df.head()

,x1,x2,y
0,0,0,1
1,1,0,1
2,0,1,1
3,1,1,0


In [65]:
#SIGMOID ACTIVATION FUNCTION#

def sigmoid(x):
    return 1/ (1 + np.exp(-x))

#SIGMOID WEIGHT UPDATE#
def sigmoid_derivative(x):
    sx = sigmoid(x)
    return sx * (1-sx)

#Initialize Random Weights for our inputs
weights = 2 * np.random.random((4,1)) - 1

#Weighted Sum
weighted_sum = np.dot(inputs, weights)

#Results of 1 Training Epoch 
activated_output = sigmoid(weighted_sum)
#True Values are [1,1,1,0]

#Calculate Error
error = correct_output - activated_output
print("\n Error:\n", error)

print("Weights:\n", weights,
      "\n \n Weighted Sum:\n", weighted_sum,
     '\n \n Activated Value for the end of 1 Epoch: \n', activated_output,
      "\n Error:\n", error,
     )

Weights:
 [[ 0.47550992]
 [-0.32009052]
 [-0.0638198 ]
 [-0.15212397]] 
 
 Weighted Sum:
 [[-0.47221449]
 [-0.21594377]] 
 
 Activated Value for the end of 1 Epoch: 
 [[0.38409224]
 [0.44622287]]


In [67]:
#Gradient Descent/Backprop
adjusted = error* sigmoid_derivative(activated_output)


 Error:
 [[ 0.61590776  0.61590776  0.61590776 -0.38409224]
 [ 0.55377713  0.55377713  0.55377713 -0.44622287]]


## Implement your own Perceptron Class and use it to classify a binary dataset: 
- [The Pima Indians Diabetes dataset](https://raw.githubusercontent.com/ryanleeallred/datasets/master/diabetes.csv) 

You may need to search for other's implementations in order to get inspiration for your own. There are *lots* of perceptron implementations on the internet with varying levels of sophistication and complexity. Whatever your approach, make sure you understand **every** line of your implementation and what its purpose is.

In [4]:
diabetes = pd.read_csv('https://raw.githubusercontent.com/ryanleeallred/datasets/master/diabetes.csv')
diabetes.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


Although neural networks can handle non-normalized data, scaling or normalizing your data will improve your neural network's learning speed. Try to apply the sklearn `MinMaxScaler` or `Normalizer` to your diabetes dataset. 

In [10]:
from sklearn.preprocessing import MinMaxScaler, Normalizer

feats = list(diabetes)[:-1]

X = ...

In [ ]:
##### Update this Class #####

class Perceptron(object):
    
    def __init__(self, niter = 10):
    self.niter = niter
    
    def __sigmoid(self, x):
        return None
    
    def __sigmoid_derivative(self, x):
        return None

    def fit(self, X, y):
    """Fit training data
    X : Training vectors, X.shape : [#samples, #features]
    y : Target values, y.shape : [#samples]
    """

    # Randomly Initialize Weights
    weights = ...

    for i in range(self.niter):
        # Weighted sum of inputs / weights

        # Activate!

        # Cac error

        # Update the Weights


    def predict(self, X):
    """Return class label after unit step"""
        return None

## Stretch Goals:

- Research "backpropagation" to learn how weights get updated in neural networks (tomorrow's lecture). 
- Implement a multi-layer perceptron. (for non-linearly separable classes)
- Try and implement your own backpropagation algorithm.
- What are the pros and cons of the different activation functions? How should you decide between them for the different layers of a neural network?